In [1]:
import requests
import pandas as pd
from glom import glom
import matplotlib.pyplot as plt
import os

## Part 0. Download box office dataset & Data pre-processing

### 0.0. Download box office dataset

Send a request to [data.gov.tw](https://data.gov.tw/) to get the download list of datasets.

In [2]:
response = requests.get("https://data.gov.tw/api/v2/rest/dataset/94224")
response_results = response.json()["result"]["distribution"]

An example of the records in `response_results`:
> ```
> [{'resourceDescription': '2018年7月30日至8月5日全國電影票房統計數據',
>    'resourceField': [{'name': '序號', 'description': ''},
>     {'name': '國別地區', 'description': ''},
>     {'name': '中文片名', 'description': ''},
>     {'name': '上映日期', 'description': ''},
>     {'name': '申請人', 'description': ''},
>     {'name': '出品', 'description': ''},
>     {'name': '上映院數', 'description': ''},
>     {'name': '銷售票數', 'description': ''},
>     {'name': '銷售金額', 'description': ''},
>     {'name': '累計銷售票數', 'description': ''},
>     {'name': '累計銷售金額', 'description': ''}],
>    'qcLevel': '',
>    'resourceFormat': 'CSV',
>    'resourceCharacterEncoding': 'UTF-8',
>    'resourceModifiedDate': '2022-09-30 14:05:28',
>    'resourceDownloadUrl': 'https://opendata.culture.tw/upload/dataSource/2018-08-09/1c3753a5-50f4-44f8-a75b-2b4d0dd2a143/69c6f154369fc266e8a3593f83d3b444.csv',
>    'resourceAmount': 241,
>    'resourceNotes': '',
>    'resourceRequestMethod': '',
>    'resourceOasUrl': '',
>    'resourceRequestParameters': []
>   },
>    ...
>  ]
>  ```

We will use the value from `resourceDownloadUrl` of `response_results` to download the box office statistics of each week.

Note that some of the datasets were stored with JSON as well as CSV. For example `response_results[96]` and `response_results[97]` represent the same dataset, one was stored with CSV, the other was stored with JSON.

In [3]:
print("response_results[96]: ", response_results[96]["resourceDescription"], "\n"
      "response_results[97]: ", response_results[97]["resourceDescription"], "\n")

response_results[96]:  2020年6月1日至2020年6月7日全國電影票房統計數據 
response_results[97]:  2020年6月1日至2020年6月7日全國電影票房統計數據JSON格式 



So we must not include the duplicated JSON datasets.

The for loop below completes the follow things: first of all, distinguish if it is not a duplicated JSON dataset, if not:
1. get the download URL of each dataset from `resourceDownloadUrl` and import the CSV with the URLs
2. store the statistical started dates of each dataset as a new column, `統計起始日`
3. concatenate all CSVs

In [4]:
urls = []
data_list = []
for ii in range(len(response_results)):
    filename = response_results[ii]["resourceDescription"]
    if "JSON格式" not in filename:
        start_date = filename.split("至")[0]
        start_date = start_date.replace("年", "/").replace("月", "/").replace("日", "")
        url = response_results[ii]["resourceDownloadUrl"]
        urls.append(url)
        table = pd.read_csv(url)
        table["統計起始日"] = start_date
        data_list.append(table)

data = pd.concat(data_list, axis = 0)

In [5]:
# data

### 0.1. Data pre-processing

Keep only the columns that will be used in the analysis later.

In [6]:
columns = ["統計起始日", "上映日期", "中文片名", "國別地區", "上映院數", "累計銷售票數", "累計銷售金額"]
df = data[columns]

Deal with the dataset bug. One record has date format error: `df["上映日期"][4829]` returns `"2019/0807"`.

In [7]:
df = df.replace({ "上映日期": "2019/0807" }, "2019/08/07")

Output to a CSV file.

In [8]:
# df.to_csv("./data/box_office_dataset.csv", index = False)

## Part 1. Import dataset

In [ ]:
df = pd.read_csv("./data/box_office_dataset.csv")

Change data types of `統計起始日` and `上映日期` to **datetime64**, and change data types of `累計銷售票數` and `累計銷售金額` to **Int64** and **Float64**, respectively.

In [9]:
df["統計起始日"] = pd.to_datetime(df["統計起始日"])
df["上映日期"] = pd.to_datetime(df["上映日期"])
df["累計銷售票數"] = pd.to_numeric(df["累計銷售票數"].str.replace(",", '')).astype("Int64")
df["累計銷售金額"] = pd.to_numeric(df["累計銷售金額"].str.replace(",", '')).astype("Float64")

In [10]:
df.dtypes

統計起始日     datetime64[ns]
上映日期      datetime64[ns]
中文片名              object
國別地區              object
上映院數               int64
累計銷售票數             Int64
累計銷售金額           Float64
dtype: object

In [11]:
# df

## Part 2. Analysis
Subjects:
1. Best box office of the year/month/week
2. 

`df` represents the original data while `df_dedup` represents the data that deduplicated `中文片名` from original data.

### 2.0. Remove duplicate movies

Remove duplicate movies, keep the record with highest `累計銷售金額`.

In [12]:
df_dedup = df.drop_duplicates(subset = "中文片名", keep = "last")

In [13]:
df_dedup

,統計起始日,上映日期,中文片名,國別地區,上映院數,累計銷售票數,累計銷售金額
17,2018-07-30,2018-07-20,鬼太鼓座,日本,4,784,166729.0
20,2018-07-30,2018-07-20,阿寶動物一族,法國,1,233,46465.0
27,2018-07-30,2018-07-13,簡單的婚禮,新加坡,1,1021,224910.0
34,2018-07-30,2018-06-29,街角的書店,英國,1,6405,1371225.0
37,2018-07-30,2018-06-22,去年冬天，與你分別,日本,1,4210,917237.0
...,...,...,...,...,...,...,...
101,2022-09-19,2017-12-29,發條橘子,英國,2,9332,2195054.0
102,2022-09-19,2014-06-13,輝耀姬物語,日本,1,810,201486.0
103,2022-09-19,2013-05-31,八月三十一日，我在奧斯陸,美國,1,3285,752357.0
104,2022-09-19,1998-10-01,海上花,中華民國,6,2458,546325.0


### 2.1. Seperate by years

In [14]:
df_2018 = df[(df["統計起始日"] > "2018-01-01") & (df["統計起始日"] <= "2018-12-31")]
df_2019 = df[(df["統計起始日"] > "2019-01-01") & (df["統計起始日"] <= "2019-12-31")]
df_2020 = df[(df["統計起始日"] > "2020-01-01") & (df["統計起始日"] <= "2020-12-31")]
df_2021 = df[(df["統計起始日"] > "2021-01-01") & (df["統計起始日"] <= "2021-12-31")]
df_2022 = df[(df["統計起始日"] > "2022-01-01") & (df["統計起始日"] <= "2022-12-31")]
print(" 2018:", len(df_2018), "records\n", "2019:", len(df_2019), "records\n", "2020:", len(df_2020), "records\n",
      "2021:", len(df_2021), "records\n", "2022:", len(df_2022), "records")

 2018: 2111 records
 2019: 4655 records
 2020: 5849 records
 2021: 3893 records
 2022: 3415 records


### 2.2. Sort by `統計起始日` and `累計銷售金額`

In [15]:
df_dedup_sort = df_dedup.sort_values(["統計起始日", "累計銷售金額"], ascending = False)

In [16]:
df_dedup_sort

,統計起始日,上映日期,中文片名,國別地區,上映院數,累計銷售票數,累計銷售金額
90,2022-09-19,2022-05-23,捍衛戰士: 獨行俠,美國,60,2660488,727991794.0
88,2022-09-19,2022-06-08,侏羅紀世界: 統霸天下,美國,7,1277581,334089877.0
93,2022-09-19,2022-05-04,奇異博士2：失控多重宇宙,美國,2,930994,244508155.0
55,2022-09-19,2022-08-13,航海王劇場版 : 紅髮歌姬,日本,96,631231,167542759.0
70,2022-09-19,2022-07-20,小小兵2:格魯的崛起,美國,43,592246,146162905.0
...,...,...,...,...,...,...,...
49,2018-07-30,2018-06-01,暴動之城,法國,1,2811,596535.0
27,2018-07-30,2018-07-13,簡單的婚禮,新加坡,1,1021,224910.0
42,2018-07-30,2018-06-15,大衛‧鮑伊is…,英國,1,433,170500.0
17,2018-07-30,2018-07-20,鬼太鼓座,日本,4,784,166729.0


## Part3. Applications

Data visualization with [bokeh](https://demo.bokeh.org/).

https://demo.bokeh.org/movies

---

## 台灣上映的電影都是來自哪些國家？

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.family'] = ['Heiti TC']

In [ ]:
country = dt4['國別地區'].value_counts()

In [ ]:
country['其他'] = country[country < 10].sum()
country_plt = country[country > 10]

In [ ]:
country_plt.plot(kind = 'pie', figsize = (6, 6), title = '在台灣上映的電影都是來自哪些國家？', 
                 autopct = '%1.0f%%', cmap = 'Set3', fontsize = 12)

---

## 過去一年內票房排行前20名的電影

In [ ]:
top20movies = dt4.sort_values('累計銷售金額', ascending=False)[:20]

In [ ]:
top20movies

---

## 2019過了三個月，目前票房排名前10名的電影

In [ ]:
dt5 = dt4[(dt4['上映日期'] > '2019-01-01') & (dt4['上映日期'] <= '2019-03-23')] #撈出今年到目前上映的電影
dt6 = dt5.sort_values(['中文片名'], ascending = False)

In [ ]:
dt7 = dt6.drop_duplicates(subset = '中文片名', keep = 'last')

In [ ]:
top10in2019 = dt7.sort_values('累計銷售金額', ascending = False)[:10]

In [ ]:
top10in2019

---

## 在台灣看一部電影平均要花多少錢？

In [ ]:
cost = dt4['累計銷售金額'].sum()
ticket = dt4['累計銷售票數'].sum()
spend = cost/ticket
print(f'在台灣，平均花 {spend:.2f} 元就可以看一部電影！')